In [1]:
# from PyQt5.uic import loadUiType
# from PyQt5 import QtWidgets
# from PyQt5 import QtCore, QtGui
import textacy
import spacy
from spacy import displacy

from Paramerator import Parameters
from Mongo import Connect
from Scraper import Scraper
from Processing import Processing
from Visualizer import Visualizer
from Stop_Handler import Stop_Handler

In [2]:
p = Parameters()
p.loader('save/params/default.params', 'params')
p.loader('dat/scraper.cfg', 'scraper')

In [3]:
connection = Connect()

In [4]:
connection.query('All', True, 48, verbose=True)

Pulling All articles from last 48 hours.
Completed pulling 111 articles.
        Latest article is from 2018-05-04 18:16:45 UTC


In [5]:
df = connection.query_df

In [ ]:
metadata = df.drop(['_id', 'text', 'spacy_chunks', 'spacy_ents', 'link', 'is article', 'post title'], axis=1)

In [ ]:
metadata = metadata.to_dict(orient='records')

In [ ]:
corpus = textacy.Corpus('en_core_web_lg', list(df.cleaned), metadatas=metadata)

In [ ]:
doc = corpus[0]

In [8]:
stops = Stop_Handler()

In [ ]:
sorted(corpus.word_freqs(as_strings=True), key=corpus.word_freqs(as_strings=True).get, reverse=True)

In [ ]:
stops.load()

In [6]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
stops.spacy_adder('en_core_web_lg', verbose=True)

In [10]:
doc = nlp(df.cleaned[0])

In [11]:
sentence = list(doc.sents)[0]

In [12]:
displacy.render(doc, style='dep', jupyter=True)

In [ ]:
for doc in corpus:
    for word_list in doc.tokenized_text:
        if "Trump" in word_list:
            print(word_list)
                

In [10]:
from Topic_Modeler import Topic_Modeler
from collections import OrderedDict

p.params.Query.Red_Blue_or_All = 'Red'
p.params.Query.Time_Frame_in_Hours = 100
topics = Topic_Modeler(connection.query_df, p.params)
topics.topic_modeler()

x = topics.model.top_topic_terms(topics.vectorizer.id_to_term, top_n=20, weights=True)

topic_list = []
for y in x:
    topic_list.append(y[1])

topic_list.sort(key=lambda dic: dic[0][1], reverse=True)

topic_dict = OrderedDict()
for x  in topic_list:
    temp_list = []
    for y in x:
        temp_list.append(y)
    
    topic_dict[x[0][0]] = temp_list

with open('data.json', 'w') as fp:
    json.dump(topic_dict, fp, indent=1)